# <center>TextRank 구현해 보기</center>

In [2]:
from newspaper import Article
from konlpy.tag import Kkma
from konlpy.tag import Twitter

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

In [4]:
kkma = Kkma()
twitter = Twitter()
stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자"
             ,"아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가", "으로", "로", "에게", "뿐이다", "의거하여", "근거하여", "입각하여", "기준으로", "예하면", "예를", "들면", "들자면", "저", "소인", "소생", "저희", "지말고", "하지마", "하지마라", "다른", "물론", "또한", "그리고", "비길수", "없다", "해서는", "안된다", "뿐만", "아니라", "만이", "아니다", "만은", "아니다", "막론하고", "관계없이", "그치지", "않다", "그러나", "그런데", "하지만", "든간에", "논하지", "않다", "따지지", "않다", "설사", "비록", "더라도", "아니면", "만", "못하다", "하는", "편이", "낫다", "불문하고", "향하여", "향해서", "향하다", "쪽으로", "틈타", "이용하여", "타다", "오르다", "제외하고", "이", "외에", "이", "밖에", "하여야", "비로소", "한다면", "몰라도", "외에도", "이곳", "여기", "부터", "기점으로", "따라서", "할", "생각이다", "하려고하다", "이리하여", "그리하여", "그렇게", "함으로써", "하지만", "일때", "할때", "앞에서", "중에서", "보는데서", "으로써", "로써", "까지", "해야한다", "일것이다", "반드시", "할줄알다", "할수있다", "할수있어", "임에", "틀림없다", "한다면", "등", "등등", "제", "겨우", "단지", "다만", "할뿐", "딩동", "댕그", "대해서", "대하여", "대하면", "훨씬", "얼마나", "얼마만큼", "얼마큼", "남짓", "여", "얼마간", "약간", "다소", "좀", "조금", "다수", "몇", "얼마", "지만", "하물며", "또한", "그러나", "그렇지만", "하지만", "이외에도", "대해", "말하자면", "뿐이다", "다음에", "반대로", "반대로", "말하자면", "이와", "반대로", "바꾸어서", "말하면", "바꾸어서", "한다면", "만약", "그렇지않으면", "까악", "툭", "딱", "삐걱거리다", "보드득", "비걱거리다", "꽈당", "응당", "해야한다", "에", "가서", "각", "각각", "여러분", "각종", "각자", "제각기", "하도록하다", "와", "과", "그러므로", "그래서", "고로", "한", "까닭에", "하기", "때문에", "거니와", "이지만", "대하여", "관하여", "관한", "과연", "실로", "아니나다를가", "생각한대로", "진짜로", "한적이있다", "하곤하였다", "하", "하하", "허허", "아하", "거바", "와", "오", "왜", "어째서", "무엇때문에", "어찌", "하겠는가", "무슨", "어디", "어느곳", "더군다나", "하물며", "더욱이는", "어느때", "언제", "야", "이봐", "어이", "여보시오", "흐흐", "흥", "휴", "헉헉", "헐떡헐떡", "영차", "여차", "어기여차", "끙끙", "아야", "앗", "아야", "콸콸", "졸졸", "좍좍", "뚝뚝", "주룩주룩", "솨", "우르르", "그래도", "또", "그리고", "바꾸어말하면", "바꾸어말하자면", "혹은", "혹시", "답다", "및", "그에", "따르는", "때가", "되어", "즉", "지든지", "설령", "가령", "하더라도", "할지라도", "일지라도", "지든지", "몇", "거의", "하마터면", "인젠", "이젠", "된바에야", "된이상", "만큼 어찌됏든", "그위에", "게다가", "점에서", "보아", "비추어", "보아", "고려하면", "하게될것이다", "일것이다", "비교적", "좀", "보다더", "비하면", "시키다", "하게하다", "할만하다", "의해서", "연이서", "이어서", "잇따라", "뒤따라", "뒤이어", "결국", "의지하여", "기대여", "통하여", "자마자", "더욱더", "불구하고", "얼마든지", "마음대로", "주저하지", "않고", "곧", "즉시", "바로", "당장", "하자마자", "밖에", "안된다", "하면된다", "그래", "그렇지", "요컨대", "다시", "말하자면", "바꿔", "말하면", "즉", "구체적으로", "말하자면", "시작하여", "시초에", "이상", "허", "헉", "허걱", "바와같이", "해도좋다", "해도된다", "게다가", "더구나", "하물며", "와르르", "팍", "퍽", "펄렁", "동안", "이래", "하고있었다", "이었다", "에서", "로부터", "까지", "예하면", "했어요", "해요", "함께", "같이", "더불어", "마저", "마저도", "양자", "모두", "습니다", "가까스로", "하려고하다", "즈음하여", "다른", "다른", "방면으로", "해봐요", "습니까", "했어요", "말할것도", "없고", "무릎쓰고", "개의치않고", "하는것만", "못하다", "하는것이", "낫다", "매", "매번", "들", "모", "어느것", "어느", "로써", "갖고말하자면", "어디", "어느쪽", "어느것", "어느해", "어느", "년도", "라", "해도", "언젠가", "어떤것", "어느것", "저기", "저쪽", "저것", "그때", "그럼", "그러면", "요만한걸", "그래", "그때", "저것만큼", "그저", "이르기까지", "할", "줄", "안다", "할", "힘이", "있다", "너", "너희", "당신", "어찌", "설마", "차라리", "할지언정", "할지라도", "할망정", "할지언정", "구토하다", "게우다", "토하다", "메쓰겁다", "옆사람", "퉤", "쳇", "의거하여", "근거하여", "의해", "따라", "힘입어", "그", "다음", "버금", "두번째로", "기타", "첫번째로", "나머지는", "그중에서", "견지에서", "형식으로", "쓰여", "입장에서", "위해서", "단지", "의해되다", "하도록시키다", "뿐만아니라", "반대로", "전후", "전자", "앞의것", "잠시", "잠깐", "하면서", "그렇지만", "다음에", "그러한즉", "그런즉", "남들", "아무거나", "어찌하든지", "같다", "비슷하다", "예컨대", "이럴정도로", "어떻게", "만약", "만일", "위에서", "서술한바와같이", "인", "듯하다", "하지", "않는다면", "만약에", "무엇", "무슨", "어느", "어떤", "아래윗", "조차", "한데", "그럼에도", "불구하고", "여전히", "심지어", "까지도", "조차도", "하지", "않도록", "않기", "위하여", "때", "시각", "무렵", "시간", "동안", "어때", "어떠한", "하여금", "네", "예", "우선", "누구", "누가", "알겠는가", "아무도", "줄은모른다", "줄은", "몰랏다", "하는", "김에", "겸사겸사", "하는바", "그런", "까닭에", "한", "이유는", "그러니", "그러니까", "때문에", "그", "너희", "그들", "너희들", "타인", "것", "것들", "너", "위하여", "공동으로", "동시에", "하기", "위하여", "어찌하여", "무엇때문에", "붕붕", "윙윙", "나", "우리", "엉엉", "휘익", "윙윙", "오호", "아하", "어쨋든", "만", "못하다    하기보다는", "차라리", "하는", "편이", "낫다", "흐흐", "놀라다", "상대적으로", "말하자면", "마치", "아니라면", "쉿", "그렇지", "않으면", "그렇지", "않다면", "안", "그러면", "아니었다면", "하든지", "아니면", "이라면", "좋아", "알았어", "하는것도", "그만이다", "어쩔수", "없다", "하나", "일", "일반적으로", "일단", "한켠으로는", "오자마자", "이렇게되면", "이와같다면", "전부", "한마디", "한항목", "근거로", "하기에", "아울러", "하지", "않도록", "않기", "위해서", "이르기까지", "이", "되다", "로", "인하여", "까닭으로", "이유만으로", "이로", "인하여", "그래서", "이", "때문에", "그러므로", "그런", "까닭에", "알", "수", "있다", "결론을", "낼", "수", "있다", "으로", "인하여", "있다", "어떤것", "관계가", "있다", "관련이", "있다", "연관되다", "어떤것들", "에", "대해", "이리하여", "그리하여", "여부", "하기보다는", "하느니", "하면", "할수록", "운운", "이러이러하다", "하구나", "하도다", "다시말하면", "다음으로", "에", "있다", "에", "달려", "있다", "우리", "우리들", "오히려", "하기는한데", "어떻게", "어떻해", "어찌됏어", "어때", "어째서", "본대로", "자", "이", "이쪽", "여기", "이것", "이번", "이렇게말하자면", "이런", "이러한", "이와", "같은", "요만큼", "요만한", "것", "얼마", "안", "되는", "것", "이만큼", "이", "정도의", "이렇게", "많은", "것", "이와", "같다", "이때", "이렇구나", "것과", "같이", "끼익", "삐걱", "따위", "와", "같은", "사람들", "부류의", "사람들", "왜냐하면", "중의하나", "오직", "오로지", "에", "한하다", "하기만", "하면", "도착하다", "까지", "미치다", "도달하다", "정도에", "이르다", "할", "지경이다", "결과에", "이르다", "관해서는", "여러분", "하고", "있다", "한", "후", "혼자", "자기", "자기집", "자신", "우에", "종합한것과같이", "총적으로", "보면", "총적으로", "말하면", "총적으로", "대로", "하다", "으로서", "참", "그만이다", "할", "따름이다", "쿵", "탕탕", "쾅쾅", "둥둥", "봐", "봐라", "아이야", "아니", "와아", "응", "아이", "참나", "년", "월", "일", "령", "영", "일", "이", "삼", "사", "오", "육", "륙", "칠", "팔", "구", "이천육", "이천칠", "이천팔", "이천구", "하나", "둘", "셋", "넷", "다섯", "여섯", "일곱", "여덟", "아홉", "령", "영"]

In [5]:
class SentenceTokenizer(object):
    def __init__(self):
        #self.url = url
        self.sentences = []
        #self.nouns = []
        
    def url2sentences(self, url):
        article = Article(url, language='ko')
        article.download()
        article.parse()
        self.sentences = kkma.sentences(article.text)
        
        for idx in range(0, len(self.sentences)):
            if len(self.sentences[idx]) <= 15:
                self.sentences[idx-1] += (' ' + self.sentences[idx])
                self.sentences[idx] = ''
        
        return self.sentences
  
    def text2sentences(self, text):
        self.sentences = kkma.sentences(text)      
        for idx in range(0, len(self.sentences)):
            if len(self.sentences[idx]) <= 10:
                self.sentences[idx-1] += (' ' + self.sentences[idx])
                self.sentences[idx] = ''
        
        return self.sentences

    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in twitter.nouns(str(sentence)) 
                                       if noun not in stopwords and len(noun) > 1]))
        
        return nouns

In [6]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
        
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return  self.graph_sentence
        
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}
    
#     def get_idx2word(self):
#         vocab = self.cnt_vec.vocabulary_
#         return {vocab[word] : word for word in vocab}
    
    def draw_graph(self, graph):
        graph = nx.from_numpy_matrix(self.graph_sentence, create_using=nx.MultiDiGraph())
        pos = nx.circular_layout(graph)
        nx.draw_circular(graph)
        labels = {i : i for i in graph.nodes()}
        nx.draw_networkx_labels(graph, pos, labels, font_size=15)
        plt.show()

In [7]:
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로 
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
            
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

In [8]:
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        
        if text[:5] in ('http:', 'https'):
            self.sentences = self.sent_tokenize.url2sentences(text)
        else:
            self.sentences = self.sent_tokenize.text2sentences(text)
        
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
                    
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        
        self.word_rank_idx =  self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
        
        
    def summarize(self, sent_num=3):
#         rank = Rank()
#         rank_idx = rank.get_ranks(self.sent_graph)
#         sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        
        
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        
        return summary
    
    def highlight_setences(self, sent_num=3, dark=0.8):
        weights = [self.sent_rank_idx[idx] for idx in self.sent_rank_idx]
        weights = (weights - min(weights))/(max(weights) - min(weights) + 1e-4)
        html = ''
        fmt = ' <span style="background-color: #{0:x}{0:x}ff">{1}</span>'
        for idx in range(len(self.sentences)):
            if idx in self.sorted_sent_rank_idx[:sent_num]:
                c = int(256*((1.-dark)*(1.-self.sent_rank_idx[idx])+dark))
            else:
                c = int(256*((1.-dark)*(1.-0)+dark))    
            html += fmt.format(c,self.sentences[idx])
        display(HTML(html))
    
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
            
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        
        return keywords

In [9]:
text = '문서에서 의미 있는 단어를 추출하고, 단어의 출현 횟수와 문서의 수에 따른 각 단어의 가중치를 산출하고, 가중치가 높은 단어들에 대하여 연관용어 집합을 구성하고 분야별 대표 색인어를 추출하고, 추출된 대표 색인어와 문서간의 유사도를 이용하여 문서를 자동으로 분류하는, 대표 색인어와 유사도를 이용한 문서 자동 분류 방법에 관한 것으로서, (a) 샘플 문서들로부터 본문 텍스트를 추출하는 단계; (b) 상기 본문 텍스트에서 형태소 분석을 하여, 불용어를 제거하고, 동의어를 처리하여, 단어들을 추출하는 단계; (c) 추출한 각 단어들에 대하여, 각 문서에 대한 단어 가중치를 계산하고, 상기 문서에 대한 단어 가중치를 평균하여, 해당 단어의 단어 가중치를 산출하는 단계; (d) 가중치가 높은 상위 단어들을 선택하고, 상기 상위 단어들로부터 각 분야별 대표 색인어를 구성하는 단계; 및, (e) 각 분야별 대표 색인어와 문서 간의 유사도를 이용하여 문서를 분야별로 분류하는 단계를 포함하는 구성을 마련한다.상기와 같은 문서 자동 분류 방법에 의하여, 분야별 대표 색인어를 추출하고 이를 이용하여 문서 유사도에 의하여 분류함으로써. 문서나 산출물들을 보다 정확하게 분야별로 분류할 수 있고, 이를 통해, 보다 정확한 분야별 검색을 제공할 수 있다.'
textrank = TextRank(text)

In [11]:
url = 'http://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=005&aid=0001064212'
textrank = TextRank(url)

In [14]:
textrank.summarize(sent_num=5)

['김 석호 서울대 사회학과 교수는 “ 일각에선 가상 화폐를 비이성적 투기라고 표현하지만 취업난과 학자금 대출 상환에 시달리는 청년들에 겐 사실상 유일한 대안 이자 가장 현실적인 선택지로 여겨 질 수 있다” 고 진단했다.',
 '윤 인진 고려대 사회학과 교수도 “ 한국 사회에선 이제 통상적인 부 축적 방식인 예금이나 부동산 투자, 주식 등을 통해서는 성공을 기대하기 어려우니 비트 코 인에 빠져들게 되는 것” 이라며 “ 로또를 사는 것과 비슷한 심리 ”라고 분석했다.',
 '투자자들의 격렬한 반응에는 분노가 서려 있다.',
 '김 지호 경북 대 심리학과 교수는 “ 분노의 목소리 중 하나는 세대 갈등” 이라며 “ 너희( 기성세대) 는 부동산으로 돈을 벌었는데 젊은 세대에게 주어진 마지막 기회를 왜 이제 와서 빼앗느냐',
 '배 규한 국민 대 사회학과 명예교수는 “ 일반 서민들은 취업될지, 취업을 한다 해도 월급으로 큰돈을 모을 수 있을지, 이런 것 들이 모두 불확실한 상태” 라며 “ 이런 사회 불안정이 극에 다다르면서 일종의 행운을 바라는 마음이 커졌다” 고 말했다.']